In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-12-27 12:16:50--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2023-12-27 12:16:50 (34.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [4]:
#read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
print("Length of the dataset in characters: ", len(text))

In [ ]:
#lets look at the first 1000 characters
print(text[:1000])

In [5]:
#here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# print(''.join(chars))
# print(vocab_size)

In [6]:
#create a mapping from characters to integers
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]   #encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])   #decoder#: take a a list of integers, output a string

# print(encode('hi there'))
# print(decode(encode('hi there')))

In [7]:
#lets now encode the entire text dataset and store it into a torch.Tensor
import torch  #we use Pytorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
# print(data.shape, data.dtype)
# print(data[:1000]) # the 1000 characters we looked at earlier will be given to the GPT look like this

In [8]:
#lets now split up the data into train and validation sets
n = int(0.9*len(data))  #first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  # print(f"when input is {context} the target is: {target}")

In [9]:
# hyperparameters
batch_size = 64 # 32 for small network # how many independent sequences will be processed in paralell?
block_size = 256 # for small netwok 8  # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500 # 300
learning_rate = 3e-4 # for small network 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 384 # for small network 32
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)

def get_batch(split):
  # generate a small batch of data of inputs x and target y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

xb, yb = get_batch('train')
# print('inputs: ')
# print(xb.shape)
# print(xb)
# print('targets: ')
# print(yb.shape)
# print(yb)

# print('----')

# for b in range(batch_size): # batch dimension
#   for t in range(block_size): # time dimension
#     context = xb[b, :t+1]
#     target = yb[b, t]
#     print(f"when input is {context.tolist()} the target: {target}")



In [ ]:
print(xb) #our input to the transformer

In [ ]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.lm_head = nn.Linear(n_embed, vocab_size)


  def forward(self, idx, targets=None):
    B, T = idx.shape

    #idx and targets are both (B,T) tensor of integers
    token_emb = self.token_embedding_table(idx) # (B,T,C)
    # pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
    x = token_emb #+ pos_emb # (B,T,vocab_size)
    logits = self.lm_head(x)  # (B,T,vocab_size)

    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T) #or targets.view(-1)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      # get the predictions
      logits, loss = self(idx)
      # focus only on the last timestep
      logits = logits[:, -1, :] # becomes (B, C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to the running sequence
      idx = torch.cat([idx, idx_next], dim=1) # (B, T+1)

    return idx

model = BigramLanguageModel()
model.to(device)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=100)[0].tolist()))

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [10]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [ ]:
# batch_size = 32

for iter in range(max_iters):

  # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  # sample a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

**The mathematical trick in self-attention**

In [ ]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B, T, C))
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1] # (t, C)
    xbow[b, t] = torch.mean(xprev, 0)

In [ ]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) ---> (B, T, C)
torch.allclose(xbow, xbow2)

In [ ]:
xbow[0], xbow2[0]

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B, T, C = 4, 8, 32  # batch, time, channel
x = torch.randn(B, T, C)

# lets see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  # (B, T, 16)
q = query(x)  # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ----> (B, T, T)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf')) # this line is specific to decoder blocks and is deleted for encoder blocks
wei = F.softmax(wei, dim=-1)
v = value(x)
out = wei @ v
# out = wei @ x

out.shape

In [ ]:
wei[0]

In [ ]:
x[0]

In [ ]:
xbow[0]

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

In [11]:
class Head(nn.Module):
  """ one head of self-attention """
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias=False)
    self.query = nn.Linear(n_embed, head_size, bias=False)
    self.value = nn.Linear(n_embed, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)  # (B, T, C)
    q = self.query(x)  # (B, T, C)
    # compute attention scores ("affinities")
    wei = q @ k.transpose(-2, -1) * C ** -0.5 # (B, T, C) @ (B, C, T) ----> (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim=-1)  # (B, T, T)
    wei = self.dropout(wei)
    # perform the weighted aggregation of the values
    v = self.value(x) # (B, T, C)
    out = wei @ v # (B, T, T) @ (B, T, C) ----> (B, T, C)
    return out

In [12]:
class MultiHeadAttention(nn.Module):
  """ multiple heads of self-attention in parallel """

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embed, n_embed)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out

In [13]:
class FeedForward(nn.Module):
  """ a simple linear layer followed by a non-linearity """

  def __init__(self, n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed, 4 * n_embed),
        nn.ReLU(),
        nn.Linear(4 * n_embed, n_embed),
        nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)

In [14]:
class Block(nn.Module):
  """ Transformer block: communication followed by computation """

  def __init__(self, n_embed, n_head):
    # n_embed: embedding dimension, n_head: the number of heads we'd like
    super().__init__()
    head_size = n_embed // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffw = FeedForward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffw(self.ln2(x))
    return x

In [15]:
class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    # self.sa_head = Head(n_embed)
    # self.sa_heads = MultiHeadAttention(4, n_embed//4) # i.e. 4 heads of 8-dimensional self-attention
    # self.ffw = FeedForward(n_embed)
    # self.blocks = nn.Sequential(
    #     Block(n_embed, n_head=4),
    #     Block(n_embed, n_head=4),
    #     Block(n_embed, n_head=4),
    #     nn.LayerNorm(n_embed)
    # )
    self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embed) # final layer norm
    self.lm_head = nn.Linear(n_embed, vocab_size)


  def forward(self, idx, targets=None):
    B, T = idx.shape

    #idx and targets are both (B,T) tensor of integers
    token_emb = self.token_embedding_table(idx) # (B,T,C)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
    x = token_emb + pos_emb # (B,T,vocab_size)
    # x = self.sa_head(x)
    # x = self.sa_heads(x) # apply one head of self-attention. (B, T, C)
    # x = self.ffw(x) # (B, T, C)
    x = self.blocks(x)  # (B, T, C)
    x = self.ln_f(x)  # (B, T, C)
    logits = self.lm_head(x)  # (B,T,vocab_size)

    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T) #or targets.view(-1)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      # crop the idx to the last block_size tokens
      idx_cond = idx[:, -block_size:]
      # get the predictions
      logits, loss = self(idx_cond)
      # focus only on the last timestep
      logits = logits[:, -1, :] # becomes (B, C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to the running sequence
      idx = torch.cat([idx, idx_next], dim=-1) # (B, T+1)

    return idx

In [16]:
model = BigramLanguageModel()
model.to(device)

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 384)
  (position_embedding_table): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffw): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affi

In [17]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [18]:
for iter in range(max_iters):

  # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  # sample a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

step 0: train loss 4.3360, val loss 4.3324
step 500: train loss 2.0122, val loss 2.0957
step 1000: train loss 1.6069, val loss 1.7830
step 1500: train loss 1.4399, val loss 1.6363
step 2000: train loss 1.3467, val loss 1.5719
step 2500: train loss 1.2840, val loss 1.5366
step 3000: train loss 1.2302, val loss 1.5084
step 3500: train loss 1.1866, val loss 1.4909
step 4000: train loss 1.1489, val loss 1.4779
step 4500: train loss 1.1115, val loss 1.4790
1.1526715755462646


In [19]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


By your father's party; steel win thou,
merch; lamentables, allow'd and flesh arm more.

PETER:
Peace, and to Haple, no: you have stand about me
As mill you be his justice of him.

PETH:
Make home good and you have no holy lace;
Whose truth vengeans of you, lords as his
favour tears for my guilty affects, while they
do near us.

Marken:
His gentleman truths sun, all men leads pon their.

Second Barina:
The wert'st they shall be strew his man? I'll not see
this of desire in farth: the maids he na


In [20]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=10000)[0].tolist()))


ING see how of Elysigh king hath grows my heart;
For like him to heir shades his curse and grows ;
And darince give him and hurl your fact.

HENRY BOLINGBROKE:
You divy, my lord, I'ld speak no gentle fault.
Cousin wish.

Second Petruch:
You shall go begg'd to thim; farewell.

HASTINGS:
I hope my lords, is but fossock, minking and you?

CLARENCE:
Second murders, I have you want.

LARTIUS:
O good in goodmandanstay;
You pray'd by Coriolanus sweet here, and all,
Never deceit to good upon your purposture,
And then longet the perpour of his name
Nor than I have a little father, and in lamons,
Then show'd musign on my breast dancing and
Stand appointed word their drunks,
And I would chare the league
To break o'er the noseperishmen hercy,
She'll appear it in love.

Second Citizen:
Farewell:
We barbend you; there clears you?

Shepherd:
Sir, no sir, in a gentleman command,
That no common lie tormurelled by
By weeping by cheeks. Fhow my crown heart.

WARWICK:
Somewife! are the matter all the nob